In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import glob, os
import datetime
%matplotlib inline

# Reading and consolidating excel files

In [4]:
# files_path = '../data/IMU Dataset'
# files = glob.glob(files_path)
# folders = ['ADLs', 'Falls', 'Near_Falls']

# df = pd.DataFrame()

# for i in range(1,11):
#     print('Subject: '+str(i))
#     for folder in folders:
#         print('-- Trial Type: '+str(folder))
#         path = files_path+'/sub'+str(i)+'/'+str(folder)+'/*.xlsx'
#         files = glob.glob(path)

#         # create a list of dataframes, one for each file, and assign a column with the name of the file
#         dfs_list = [pd.read_excel(fp).assign(FileName = os.path.basename(fp)) for fp in files]

#         df_temp = pd.DataFrame()
#         df_temp = pd.concat(dfs_list)
#         df_temp['subject'] = i
#         df_temp['trial_type'] = folder
        
#         df = df.append(df_temp)

In [6]:
# save the dataframe in a Pickle file
# pickle.dump(df, open("dataset_raw.p", "wb"))

# load the dataframe
df = pickle.load(open("../data/dataset_raw.p", "rb"))

## Including implicit features

In [7]:
df['trial_type'].value_counts()

ADLs          498810
Falls         403402
Near_Falls    288157
Name: trial_type, dtype: int64

In [8]:
# 'Trial Type' is not ordinal, so we will need dummy variables

# extraction of the trial number from the file name
# Not Ordinal: necessary dummy variables
df['trial_num'] = df['FileName'].apply(lambda x: x.replace('.xlsx','').split('_')[-1])

# extraction of the trial subtype (slip, trip, pick object from ground, etc) from the file name
# Necessary dummy variables
df['trial_subtype'] = df['FileName'].apply(lambda x: x.split('_')[1])

# column with the real date and time of the measurement
# according to the README.txt, the Time column is the number of microseconds from 01/01/1970
initial = datetime.datetime(1970,1,1,0,0,0)
df['time_datetime'] = df.Time.apply(lambda x: (initial + datetime.timedelta(microseconds=x)))
# creating the column 'Time_datetime' in a pandas datetime format
df['time_datetime'] = df.time_datetime.apply(lambda t: pd.datetime(t.year,t.month,t.day,t.hour,t.minute,t.second,t.microsecond))

###### The first 3 characters of the file name seems to be a supercategory of the trial_subtype.
###### This way, I believe we can discard it and use just trial_subtype
df['target'] = np.where(df['trial_type'] == 'Falls', 1, 0)
df.head()

,Time,r.ankle Acceleration X (m/s^2),r.ankle Acceleration Y (m/s^2),r.ankle Acceleration Z (m/s^2),r.ankle Angular Velocity X (rad/s),r.ankle Angular Velocity Y (rad/s),r.ankle Angular Velocity Z (rad/s),r.ankle Magnetic Field X (uT),r.ankle Magnetic Field Y (uT),r.ankle Magnetic Field Z (uT),...,waist Magnetic Field X (uT),waist Magnetic Field Y (uT),waist Magnetic Field Z (uT),FileName,subject,trial_type,trial_num,trial_subtype,time_datetime,target
0,1304799112429680,-9.691464,-0.240769,-1.514349,0.218779,0.055136,-0.011516,33.449158,19.266598,15.818048,...,45.724129,-24.729599,-24.925210,JXL_SQ_trial1.xlsx,1,ADLs,trial1,SQ,2011-05-07 20:11:52.429680,0
1,1304799112437500,-9.698196,-0.245077,-1.509888,0.213975,0.059683,-0.016308,33.449158,19.266598,15.818048,...,43.054999,-27.175334,-17.818824,JXL_SQ_trial1.xlsx,1,ADLs,trial1,SQ,2011-05-07 20:11:52.437500,0
2,1304799112445310,-9.707324,-0.240524,-1.503184,0.215362,0.048608,-0.008371,33.925814,18.780263,16.358234,...,35.924348,-24.854392,-22.314215,JXL_SQ_trial1.xlsx,1,ADLs,trial1,SQ,2011-05-07 20:11:52.445310,0
3,1304799112453120,-9.700565,-0.238417,-1.503189,0.212007,0.051853,-0.006869,33.925814,18.780263,16.358234,...,40.471139,-33.319708,-20.664150,JXL_SQ_trial1.xlsx,1,ADLs,trial1,SQ,2011-05-07 20:11:52.453120,0
4,1304799112460930,-9.698231,-0.242868,-1.507656,0.217024,0.055225,-0.008403,33.436905,18.789406,16.859036,...,45.633920,-24.349124,-20.064366,JXL_SQ_trial1.xlsx,1,ADLs,trial1,SQ,2011-05-07 20:11:52.460930,0


In [9]:
# Reorganizing column names

In [10]:
df = df[['Time', 'time_datetime',  'subject', 'trial_type',
       'trial_num', 'trial_subtype', 'r.ankle Acceleration X (m/s^2)',
       'r.ankle Acceleration Y (m/s^2)', 'r.ankle Acceleration Z (m/s^2)',
       'r.ankle Angular Velocity X (rad/s)',
       'r.ankle Angular Velocity Y (rad/s)',
       'r.ankle Angular Velocity Z (rad/s)', 'r.ankle Magnetic Field X (uT)',
       'r.ankle Magnetic Field Y (uT)', 'r.ankle Magnetic Field Z (uT)',
       'l.ankle Acceleration X (m/s^2)', 'l.ankle Acceleration Y (m/s^2)',
       'l.ankle Acceleration Z (m/s^2)', 'l.ankle Angular Velocity X (rad/s)',
       'l.ankle Angular Velocity Y (rad/s)',
       'l.ankle Angular Velocity Z (rad/s)', 'l.ankle Magnetic Field X (uT)',
       'l.ankle Magnetic Field Y (uT)', 'l.ankle Magnetic Field Z (uT)',
       'r.thigh Acceleration X (m/s^2)', 'r.thigh Acceleration Y (m/s^2)',
       'r.thigh Acceleration Z (m/s^2)', 'r.thigh Angular Velocity X (rad/s)',
       'r.thigh Angular Velocity Y (rad/s)',
       'r.thigh Angular Velocity Z (rad/s)', 'r.thigh Magnetic Field X (uT)',
       'r.thigh Magnetic Field Y (uT)', 'r.thigh Magnetic Field Z (uT)',
       'l.thigh Acceleration X (m/s^2)', 'l.thigh Acceleration Y (m/s^2)',
       'l.thigh Acceleration Z (m/s^2)', 'l.thigh Angular Velocity X (rad/s)',
       'l.thigh Angular Velocity Y (rad/s)',
       'l.thigh Angular Velocity Z (rad/s)', 'l.thigh Magnetic Field X (uT)',
       'l.thigh Magnetic Field Y (uT)', 'l.thigh Magnetic Field Z (uT)',
       'head Acceleration X (m/s^2)', 'head Acceleration Y (m/s^2)',
       'head Acceleration Z (m/s^2)', 'head Angular Velocity X (rad/s)',
       'head Angular Velocity Y (rad/s)', 'head Angular Velocity Z (rad/s)',
       'head Magnetic Field X (uT)', 'head Magnetic Field Y (uT)',
       'head Magnetic Field Z (uT)', 'sternum Acceleration X (m/s^2)',
       'sternum Acceleration Y (m/s^2)', 'sternum Acceleration Z (m/s^2)',
       'sternum Angular Velocity X (rad/s)',
       'sternum Angular Velocity Y (rad/s)',
       'sternum Angular Velocity Z (rad/s)', 'sternum Magnetic Field X (uT)',
       'sternum Magnetic Field Y (uT)', 'sternum Magnetic Field Z (uT)',
       'waist Acceleration X (m/s^2)', 'waist Acceleration Y (m/s^2)',
       'waist Acceleration Z (m/s^2)', 'waist Angular Velocity X (rad/s)',
       'waist Angular Velocity Y (rad/s)', 'waist Angular Velocity Z (rad/s)',
       'waist Magnetic Field X (uT)', 'waist Magnetic Field Y (uT)',
       'waist Magnetic Field Z (uT)', 'FileName', 'target']]

In [11]:
df.head(2)

,Time,time_datetime,subject,trial_type,trial_num,trial_subtype,r.ankle Acceleration X (m/s^2),r.ankle Acceleration Y (m/s^2),r.ankle Acceleration Z (m/s^2),r.ankle Angular Velocity X (rad/s),...,waist Acceleration Y (m/s^2),waist Acceleration Z (m/s^2),waist Angular Velocity X (rad/s),waist Angular Velocity Y (rad/s),waist Angular Velocity Z (rad/s),waist Magnetic Field X (uT),waist Magnetic Field Y (uT),waist Magnetic Field Z (uT),FileName,target
0,1304799112429680,2011-05-07 20:11:52.429680,1,ADLs,trial1,SQ,-9.691464,-0.240769,-1.514349,0.218779,...,-0.122713,2.294797,0.150650,1.112724,-0.005126,45.724129,-24.729599,-24.925210,JXL_SQ_trial1.xlsx,0
1,1304799112437500,2011-05-07 20:11:52.437500,1,ADLs,trial1,SQ,-9.698196,-0.245077,-1.509888,0.213975,...,-0.131644,2.297199,0.150732,1.119032,-0.005089,43.054999,-27.175334,-17.818824,JXL_SQ_trial1.xlsx,0


In [13]:
# save the dataframe in a Pickle file
pickle.dump(df, open("../data/dataset_consolidated.p", "wb"))

<hr>

### Metadata

To facilitate the data management, we'll store meta-information about the variables in a DataFrame. This will be helpful when we want to select specific variables for analysis, visualization, modeling, ...

Concretely we will store:

* Body Location: r.ankle, l.ankle, r.thigh, l.thigh, head, sternum, waist
* Axes: X, Y, Z
* Unit: m/s^2, rad/s, uT
* Measurements: aceleration, angular Velocity, magnetic field
* dtype: int, float, str

In [14]:
data = []
for f in df.columns:

    measure = ''
    # Defining the measure
    if 'Acceleration' in f:
        measure = 'acceleration'
    elif 'Angular Velocity' in f:
        measure = 'angular velocity'
    elif 'Magnetic Field' in f:
        measure = 'magnetic field'
         
    # Defining the body location
    body_location = ''
    if 'r.ankle' in f:
        body_location = 'r.ankle'
    elif 'l.ankle' in f:
        body_location = 'l.ankle'
    elif 'r.thigh' in f:
        body_location = 'r.thigh'
    elif 'l.thigh' in f:
        body_location = 'l.thigh'
    elif 'head' in f:
        body_location = 'head'
    elif 'sternum' in f:
        body_location = 'sternum'
    elif 'waist' in f:
        body_location = 'waist'

    axis = ''
    # Defining the Axes
    if ' X ' in f:
        axis = 'X'
    elif ' Y ' in f:
        axis = 'Y'
    elif ' Z ' in f:
        axis = 'Z'       
        
    unit = ''
    # Defining the Axes
    if 'rad/s' in f:
        unit = 'rad/s'
    elif 'm/s^2' in f:
        unit = 'm/s^2'
    elif 'uT' in f:
        unit = 'uT'           
    
    # Defining the data type 
    dtype = df[f].dtype
    
    # Creating a Dict that contains all the metadata for the variable
    f_dict = {
        'varname': f,
        'body_location': body_location,
        'axis': axis,
        'unit': unit,
        'measure': measure,
        'dtype': dtype
    }
    data.append(f_dict)
    
meta = pd.DataFrame(data, columns=['varname', 'body_location', 'axis', 'unit', 'measure', 'dtype'])
meta.set_index('varname', inplace=True)

In [16]:
# save the dataframe
pickle.dump(meta, open("../data/metadata.p", "wb"))